# MVP-L: BEV Occupancy 생성

이 노트북은 포인트 클라우드에서 ground를 분리하고 BEV (Bird's Eye View) occupancy grid를 생성합니다.

## 목표
- Ground 분리 (RANSAC plane fitting)
- BEV occupancy grid 생성
- 주행가능영역 시각화

In [ ]:
# Cell 1: 라이브러리 임포트
import numpy as np
import open3d as o3d
from sklearn.linear_model import RANSACRegressor
from sklearn.preprocessing import PolynomialFeatures
import matplotlib.pyplot as plt
from pathlib import Path

print("라이브러리 로드 완료")

In [ ]:
# Cell 2: RANSAC을 사용한 Ground 분리
def separate_ground_ransac(points, distance_threshold=0.3, max_iterations=1000):
    """
    RANSAC을 사용하여 ground plane 분리
    
    Args:
        points: (N, 3) 포인트 클라우드
        distance_threshold: inlier로 간주할 최대 거리
        max_iterations: 최대 반복 횟수
    
    Returns:
        ground_points: (M, 3) ground 포인트
        non_ground_points: (N-M, 3) non-ground 포인트
        plane_model: [a, b, c, d] where ax + by + cz + d = 0
    """
    # Open3D 포인트 클라우드 생성
    pcd = o3d.geometry.PointCloud()
    pcd.points = o3d.utility.Vector3dVector(points)
    
    # RANSAC plane segmentation
    plane_model, inliers = pcd.segment_plane(
        distance_threshold=distance_threshold,
        ransac_n=3,
        num_iterations=max_iterations
    )
    
    # Ground와 non-ground 분리
    ground_indices = np.array(inliers)
    non_ground_indices = np.setdiff1d(np.arange(len(points)), ground_indices)
    
    ground_points = points[ground_indices]
    non_ground_points = points[non_ground_indices]
    
    return ground_points, non_ground_points, plane_model

print("Ground 분리 함수 정의 완료")

In [ ]:
# Cell 3: BEV Occupancy Grid 생성
def create_bev_occupancy_grid(non_ground_points, grid_size=0.2, 
                               x_range=(-50, 50), y_range=(-50, 50),
                               height_threshold=0.2):
    """
    BEV occupancy grid 생성
    
    Args:
        non_ground_points: (N, 3) non-ground 포인트
        grid_size: 그리드 셀 크기 (미터)
        x_range: (x_min, x_max) 범위
        y_range: (y_min, y_max) 범위
        height_threshold: occupancy 판단을 위한 최소 높이
    
    Returns:
        occupancy_grid: (H, W) 2D 배열, True=occupied, False=free
        grid_coords: 그리드 좌표 정보
    """
    # 그리드 크기 계산
    x_min, x_max = x_range
    y_min, y_max = y_range
    grid_width = int((x_max - x_min) / grid_size)
    grid_height = int((y_max - y_min) / grid_size)
    
    # 그리드 인덱스 계산
    x_indices = ((non_ground_points[:, 0] - x_min) / grid_size).astype(np.int32)
    y_indices = ((non_ground_points[:, 1] - y_min) / grid_size).astype(np.int32)
    
    # 범위 내 포인트만 사용
    valid_mask = ((x_indices >= 0) & (x_indices < grid_width) &
                  (y_indices >= 0) & (y_indices < grid_height))
    x_indices = x_indices[valid_mask]
    y_indices = y_indices[valid_mask]
    z_values = non_ground_points[valid_mask, 2]
    
    # Occupancy grid 생성 (높이 기반)
    occupancy_grid = np.zeros((grid_height, grid_width), dtype=bool)
    height_map = np.zeros((grid_height, grid_width))
    
    for x_idx, y_idx, z in zip(x_indices, y_indices, z_values):
        if z > height_threshold:  # 높이가 임계값보다 크면 occupied
            occupancy_grid[y_idx, x_idx] = True
        height_map[y_idx, x_idx] = max(height_map[y_idx, x_idx], z)
    
    grid_coords = {
        'x_min': x_min, 'x_max': x_max,
        'y_min': y_min, 'y_max': y_max,
        'grid_size': grid_size,
        'grid_width': grid_width,
        'grid_height': grid_height
    }
    
    return occupancy_grid, height_map, grid_coords

print("BEV occupancy grid 생성 함수 정의 완료")

In [ ]:
# Cell 4: BEV 시각화
def visualize_bev_occupancy(occupancy_grid, height_map, grid_coords, title="BEV Occupancy"):
    """
    BEV occupancy grid 시각화
    """
    fig, axes = plt.subplots(1, 2, figsize=(16, 8))
    
    # Occupancy grid (binary)
    ax = axes[0]
    ax.imshow(occupancy_grid, origin='lower', cmap='RdYlGn_r', 
              extent=[grid_coords['x_min'], grid_coords['x_max'],
                     grid_coords['y_min'], grid_coords['y_max']])
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(f'{title} - Occupancy Grid')
    ax.grid(True, alpha=0.3)
    
    # Height map
    ax = axes[1]
    im = ax.imshow(height_map, origin='lower', cmap='jet',
                  extent=[grid_coords['x_min'], grid_coords['x_max'],
                         grid_coords['y_min'], grid_coords['y_max']])
    ax.set_xlabel('X (m)')
    ax.set_ylabel('Y (m)')
    ax.set_title(f'{title} - Height Map')
    ax.grid(True, alpha=0.3)
    plt.colorbar(im, ax=ax, label='Height (m)')
    
    plt.tight_layout()
    plt.show()
    return fig

print("시각화 함수 정의 완료")

In [ ]:
# Cell 5: 주행가능영역 추정
def estimate_drivable_area(occupancy_grid, grid_coords, expansion_radius=2.0):
    """
    주행가능영역 추정 (occupied 영역 주변 제외)
    
    Args:
        occupancy_grid: (H, W) occupancy grid
        grid_coords: 그리드 좌표 정보
        expansion_radius: occupied 영역 주변 제외 반경 (미터)
    
    Returns:
        drivable_grid: (H, W) 주행가능영역 (True=drivable)
    """
    from scipy.ndimage import binary_dilation
    
    # 미터를 그리드 단위로 변환
    expansion_pixels = int(expansion_radius / grid_coords['grid_size'])
    
    # Occupied 영역 확장 (안전 마진 추가)
    expanded_occupied = binary_dilation(occupancy_grid, 
                                       structure=np.ones((expansion_pixels*2+1, expansion_pixels*2+1)))
    
    # 주행가능영역 = occupied가 아닌 영역
    drivable_grid = ~expanded_occupied
    
    return drivable_grid

print("주행가능영역 추정 함수 정의 완료")

In [ ]:
# Cell 6: 통합 파이프라인 테스트
# 포인트 클라우드 로드 및 BEV 생성
project_root = Path.cwd().parent.parent
velodyne_dir = project_root / "dataset" / "training" / "velodyne"

if velodyne_dir.exists():
    bin_files = sorted(velodyne_dir.glob("*.bin"))
    if bin_files:
        # 포인트 클라우드 로드
        points = np.fromfile(str(bin_files[0]), dtype=np.float32).reshape(-1, 4)[:, :3]
        print(f"포인트 개수: {len(points):,}")
        
        # Ground 분리
        ground_points, non_ground_points, plane_model = separate_ground_ransac(points)
        print(f"Ground 포인트: {len(ground_points):,}")
        print(f"Non-ground 포인트: {len(non_ground_points):,}")
        print(f"Plane model: {plane_model}")
        
        # BEV occupancy grid 생성
        occupancy_grid, height_map, grid_coords = create_bev_occupancy_grid(
            non_ground_points, grid_size=0.2, x_range=(-50, 50), y_range=(-50, 50)
        )
        print(f"Occupancy grid shape: {occupancy_grid.shape}")
        print(f"Occupied cells: {np.sum(occupancy_grid)}")
        
        # 시각화
        fig = visualize_bev_occupancy(occupancy_grid, height_map, grid_coords,
                                     title=f"Sample: {bin_files[0].name}")
        
        # 주행가능영역 추정
        drivable_grid = estimate_drivable_area(occupancy_grid, grid_coords)
        print(f"Drivable cells: {np.sum(drivable_grid)}")
    else:
        print("포인트 클라우드 파일을 찾을 수 없습니다.")
else:
    print("Velodyne 디렉토리를 찾을 수 없습니다.")